# Наука о данных. Финальный проект

*2023-2024 учебный год*

**Автор:** *Лосевской Артём*

# Описание проекта

Кратко скажу здесь, что проект посвящён анализу спортивного плавания в российских регионах. Более подробную информацию можно найти в файле README.

Используемые библиотеки и инструменты:

1. Pandas
2. Scrapy
3. Re
4. Matplotlib
5. Numpy
6. Scikit-learn
7. Geopandas и Folium
8. SQL (SQLAlchemy)

# Часть 0. Необходимые приготовления

Импортируем библиотеки:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Часть 1. Парсинг данных

Соберем все данные по количеству бассейнов в каждом регионе. Для этого воспользуюсь сайтом Всероссийской Федерации Плавания. Скрепинг осуществлен с помощью scrapy — весь код в отдельном файле. 

Данные немного устаревшие (2019 год), но более свежих достоверных сведений по количеству бассейнов, в которых осуществляется сопртивная подготвока, не найти. Итого на 2019 год имеем данные по 85 субъектам:

In [ ]:
pools = pd.read_json("pools_2019.json")
pools

In [ ]:
pools['region'].tolist()

# Часть 2. Работа с датасетами

Загрузим датасет с населением России и полигонами для карты

In [ ]:
import geopandas as gpd
regions_data = gpd.read_file("russia_regions_simplified.json")
regions_data

In [ ]:
regions_data.plot(column = 'federal_district', legend = True, cmap = 'viridis', figsize=[16, 9])

В датасете я обнаружил 2 проблемы. 
- Во-первых, на нём есть Крымский федеральный округ (такой просуществовал в 2014-2016 годах). Севастополь и республику Крым нужно отнести к Южному ФО.
- Во-вторых, часть Чукотки оторвана от остальной России. Карту хорошо бы привести в нужную географическую систему отсчета. 

Займёмся этим:

In [ ]:
regions_data[regions_data['federal_district'] == 'Крымский']

In [ ]:
li = regions_data[regions_data['federal_district'] == 'Крымский'].index.tolist()
regions_data.loc[li, 'federal_district'] = 'Южный'

regions_data_fot_plotting = regions_data.to_crs('EPSG:32646') #Исходная система отсчёта понадобится позже, когда начну работать с folium.

#Пробовал и просто перейти к этой системе отсчёта, но при работе с Folium возникали проблемы с геометрией

In [ ]:
regions_data_fot_plotting.plot(column = 'region', legend = True, figsize=[16, 9])

In [ ]:
regions_data_fot_plotting.plot(column = 'federal_district', legend = True, cmap = 'viridis', figsize=[16, 9])

In [ ]:
regions_data

Карта не пострадала. Другая проблема — несоответствие имён некоторых регионов с датасетом, содержащим бассейны. Для их объединения надо бы привести всё к одному виду. 

Некоторые регионы называются в датасетах по-разному. Например, "Город федерального значения Москва" Придётся переименовать их вручную:

Избавлюсь от Город фед.значения в pools; заменю в regions_data и pools автономный округ (в обоих) и автономная область (это в regions_data) на АО. Югра — вручную.

In [ ]:
pools['region'] = pools['region'].str.replace('Город федерального значения ', '')
pools['region'] = pools['region'].str.replace(' автономный округ', ' АО')
pools

In [ ]:
regions_data['region'] = regions_data['region'].str.replace(' автономный округ', ' АО')
regions_data['region'] = regions_data['region'].str.replace(' автономная область', ' АО')
regions_data['region'] = regions_data['region'].str.replace('Ханты-Мансийский АО — Югра', 'Ханты-Мансийский АО')
regions_data[regions_data['region'].str.contains('АО')]

In [ ]:
regions_pools = pd.merge(pools, regions_data, left_on = 'region', right_on = 'region', how = "inner")
regions_pools[regions_pools['region'].str.contains('АО')]

In [ ]:
regions_pools

84, нужно 85. Еще есть проблема с Северной Осетией. Не выяснил, в чём именно. Просто заменю в pools:

In [ ]:
regions_pools_test = pd.merge(pools, regions_data, left_on = 'region', right_on = 'region', how = "left")
regions_pools_test[regions_pools_test['population'].isnull()]

In [ ]:
pools[pools['region'] == 'Республика Северная Осетия — Алания']

In [ ]:
regions_data[regions_data['region'] == 'Республика Северная Осетия — Алания']

In [ ]:
#need_replacement = pools[pools['region'].str.contains('Северная Осетия')]
pools.iloc[41, 0] = 'Республика Северная Осетия — Алания'

In [ ]:
#pools['region'] = pools['region'].str.replace(' автономный округ', 'jct')
regions_pools = pd.merge(regions_data, pools, left_on = 'region', right_on = 'region', how = "inner")
regions_pools

Теперь всё в порядке: имеем все (на 2019) 85 регионов.

# Часть 3.1. Анализ количества бассейнов в регионах

## Гипотеза

Первая гипотеза, которая приходит на ум после взгляда на имеющиеся данные (да в общем-то и без них тоже), это то, что количество бассейнов в регионе положительно зависит от численности населения.

Попробуем для начала отсортировать регионы по общему количеству бассейнов в них, а затем — по количеству пятидесяти метровых бассейнов.

## Визуализация

Имеем две переменные: население региона и количество бассейнов в нём. Независимой переменной будем считать население, а прогнозируемой переменной — количество бассейнов (общее количество, а также количество 50-метровых). В обоих случаях попробуем также задать линию тренда, то есть апроксимировать полиномом первой степени.

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
x = regions_pools['population'] / 1000000
y1, y2 = regions_pools['total'], regions_pools['50m']

ax1.scatter(x, y1, color = 'b')
ax1.set_xlabel('Население, млн', fontsize = 12)
ax1.set_ylabel('Количество бассейнов', fontsize = 12)
ax1.set_title('Scatter plot: население региона — общее число бассейнов', weight = 'bold', fontsize = 14)
ax1.set_facecolor('whitesmoke')
ax1.grid(True)
z = np.polyfit(regions_pools['population'] / 1000000, regions_pools['total'], 1)
p = np.poly1d(z)
ax1.plot(x, p(x), color = "purple")

ax2.scatter(x, y2, color = 'b')
ax2.set_xlabel('Население, млн', fontsize = 12)
ax2.set_ylabel('Количество "полтинников"', fontsize = 12)
ax2.set_title('Scatter plot: население региона — общее число бассейнов', weight = 'bold', fontsize = 14)
ax2.set_facecolor('whitesmoke')
ax2.grid(True)
z2 = np.polyfit(regions_pools['population'] / 1000000, regions_pools['50m'], 1)
p2 = np.poly1d(z2)
ax2.plot(x, p2(x), color = "purple")


Москва и Московская область очень отрываются от остальных по населению. А если их убрать? 

In [ ]:
#Применим группировку по населению: отбросим регионы с населением более 6 млн человек. 
regions_pools_dropMoscowAndMO = regions_pools.loc[regions_pools['population'] <= 6000000]
#Так уж совпадает, что таких регионов всего два — Москва и Мос. область.
regions_pools_dropMoscowAndMO.sort_values('population', ascending = False)

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
x = regions_pools_dropMoscowAndMO['population'] / 1000000
y1, y2 = regions_pools_dropMoscowAndMO['total'], regions_pools_dropMoscowAndMO['50m']

ax1.scatter(x, y1, color = 'b')
ax1.set_xlabel('Население, млн', fontsize = 12)
ax1.set_ylabel('Количество бассейнов', fontsize = 12)
ax1.set_title('Scatter plot: население региона — общее число бассейнов', weight = 'bold', fontsize = 14)
ax1.set_facecolor('whitesmoke')
ax1.grid(True)
z = np.polyfit(regions_pools['population'] / 1000000, regions_pools['total'], 1)
p = np.poly1d(z)
ax1.plot(x, p(x), color = "purple", linewidth = 2, linestyle = "dashed")

ax2.scatter(x, y2, color = 'b')
ax2.set_xlabel('Население, млн', fontsize = 12)
ax2.set_ylabel('Количество "полтинников"', fontsize = 12)
ax2.set_title('Scatter plot: население региона — общее число бассейнов', weight = 'bold', fontsize = 14)
ax2.set_facecolor('whitesmoke')
ax2.grid(True)
z2 = np.polyfit(regions_pools['population'] / 1000000, regions_pools['50m'], 1)
p2 = np.poly1d(z2)
ax2.plot(x, p2(x), color = "purple", linestyle = "dashed")


Вполне ожидаемо видим явный тренд на увеличение числа бассейнов по мере роста населения.

## Корелляции числа бассейнов и населения

Посмотрим, какова корелляция переменных:

In [ ]:
regions_pools.sort_values('total', ascending = False)

In [ ]:
regions_pools.sort_values('50m', ascending = False)

In [ ]:
correlation = regions_pools['total'].corr(regions_pools['population'])
print(correlation)

In [ ]:
correlation_50m = regions_pools['50m'].corr(regions_pools['population'])
print(correlation_50m)

Видим, что общая численность населения в регионе сильно положительно кореллирует с общим количеством бассейнов в нём, и несколько хуже — с количеством "полтинников".

## Предсказательная модель

Попробуем построить предсказательную модель числа бассейнов в регионе по количеству жителей в регионе. Для этого применим линейную регресиию и самый базовый метод оценки коэффициентов — метод наименьших квадратов.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

X = np.array(regions_pools['population']).reshape(-1, 1)
y = np.array(regions_pools['total'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print("Mean squared error (MSE):  %.2f" % mean_squared_error(y_test, y_pred))
print("Mean absolute error (MAE):  %.2f" % mean_absolute_error(y_test, y_pred))
print("Coefficient of determination (R^2): %.2f" % r2_score(y_test, y_pred))

plt.scatter(X_test, y_test, color="black")
plt.plot(X_test, y_pred, color="blue", linewidth=2)
plt.xlabel("Население, млн")
plt.ylabel("Количество бассейнов")
plt.title("Реальные данные и прогноз")
plt.show()

Проведём аналогичную процедуру, но теперь без Москвы и МО (МО, как видно из графика, была в тестируемой выборке):

In [ ]:
X = np.array(regions_pools_dropMoscowAndMO['population']).reshape(-1, 1)
y = np.array(regions_pools_dropMoscowAndMO['total'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print("Mean squared error (MSE):  %.2f" % mean_squared_error(y_test, y_pred))
print("Mean absolute error (MAE):  %.2f" % mean_absolute_error(y_test, y_pred))
print("Coefficient of determination (R^2): %.2f" % r2_score(y_test, y_pred))

plt.scatter(X_test, y_test, color="black")
plt.plot(X_test, y_pred, color="blue", linewidth=2)
plt.xlabel("Население, млн")
plt.ylabel("Количество бассейнов")
plt.title("Реальные данные и прогноз")
plt.show()

Коэффициент детерминации заметно снизился, что несколько неожиданный результат. С другой стороны, Москва и МО были близки к прогнозируемым значениям (в отличии от крайних точек справа для датасета без Москвы и МО), поэтому такой результат тоже можно было допустить.

## Карта 1

Попробуем нанести на карту имеющуюся у нас информацию по числу бассейнов. Чем больше в регионе общее бассейнов, тем более тёплым будет цвет региона. 

In [ ]:
regions_pools_for_plotting = regions_pools.to_crs('EPSG:32646')

fig, ax = plt.subplots(figsize=[16, 9])
ax = regions_pools_for_plotting.plot(ax = ax, column = 'total', legend = True, cmap = 'coolwarm', figsize=[16, 9])

plt.title('Количество бассейнов в регионах', fontsize = 22)
ax.set_axis_off()

plt.show()

Откровенно говоря, информацию из такой карты почерпнуть сложно. Воспользуюсь folium

## Карта 2

Создадим с помощью folium интерактивную карту с палитрой цветов, что будет раскрашивать регион в зависимости от общего количества бассейнов. Также вынесем в посдказки информацию как об общем количестве бассейнов, так и о 50-метровых бассейнах в выбранном регионе.

In [ ]:
#выгружаю датасет локально, чтобы впоследствии сделать демонстрацию карты в Streamlit

regions_pools.to_file("regions_pools.json", driver="GeoJSON")

In [ ]:
import folium
import branca.colormap as cm

m = folium.Map(location=[64.6863136, 97.7453061], zoom_start=3)

max_count = regions_pools['total'].max()
min_count = regions_pools['total'].min()
colormap = cm.linear.YlGnBu_09.scale(min_count, max_count)
colormap.add_to(m)

def style_function(feature):
    regions_pools = feature['properties']['total']
    return {
        'fillColor': colormap(regions_pools),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

folium.GeoJson(
    regions_pools,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['region', 'total', '50m'],
        aliases=['Регион:', 'Количество бассейнов:', 'Количество "полтинников":'],
        localize=True
    )
).add_to(m)

m

# Часть 3.2. Анализ развития спортивного плавания в регионах

## Новый показатель и гипотеза

Знать, сколько бассейнов в регионе, хорошо и приятно. Однако куда интереснее было бы узнать, насколько разивто в регионе спортивное плавание. Здесь судить по одному лишь количеству бассейнов может быть некорректно.

До сих пор, собирая данные и проводя анализ количества бассейнов, я отдельно выделял "длинные" бассейны. Причина в том, что для нашего нынешнего анализа — анализа по части развитии спортивного плавания в регионе — бассейны длиной 50 метров могут быть более показтельны, потому что Олимпийские Игры проводится именно в 50-метровых бассейнах.

Добавим ещё один показатель: сколько жителей региона приходится на один бассейн. Своего рода плотность населения по бассейнам. И ещё один: сколько жителей региона приходится на один 50-метровый бассейн.

In [ ]:
pd.options.display.float_format = '{:20.2f}'.format
regions_pools['pools_density'] = regions_pools['population'] / regions_pools['total']
regions_pools['50m_pools_density'] = regions_pools['population']  / regions_pools['50m']
regions_pools.replace(np.inf, 0, inplace = True)
regions_pools

*Впрочем, я передумал*. У такого показателя есть ряд недостатков.

Поступим наоборот: поделим число бассейнов на население. Получим что-то вроде плотности бассейнов. 

Такой показатель легче интерпретировать, но нужно задуматься о том, как его нормировать, ведь числа очень маленькие (действительно: смысл данного показтеля в том, чтобы показывать, сколько бассейнов приходится на 1 жителя региона). Например, домножим на 1.000.000, и тогда получим число, показывающее, сколько бассейнов приходится на 1 млн жителей в данном регионе.

In [ ]:
regions_pools['pools_density'] = round(regions_pools['total'] / regions_pools['population'] * 100000, 2)
regions_pools['50m_pools_density'] = round(regions_pools['50m'] / regions_pools['population'] * 100000, 2)
regions_pools

Интуиция подсказывает, что чем больше это число, тем, должно быть, более комфортные условия для плавания и для его развития созданы в регионе. 
Попробуем посмотреть, где плотность бассейнов большего всего:

In [ ]:
regions_pools.sort_values(by = 'pools_density', ascending = False)

In [ ]:
regions_pools.sort_values(by = '50m_pools_density', ascending = False)

Итак, у нас появилась новая метрика, но собственно показателя, прямо говорящего о спортивных достижениях региона в плавании у нас пока не было. Пора это исправить.

## Ещё парсинг: Мастера спорта России по плаванию

В качестве мерила успехов региона в спортивном плавании примем количество Мастеров спорта по плаванию в этом регионе. 

Для этого запарсим сайт, собирающий статистику и информацию по развитию спорта в России. Код паука scrapy, опять-таки, будет отдельным файлом, а здесь я просто выгружу полученный датасет:

In [ ]:
masters = pd.read_json("MS_quant.json")
masters

In [ ]:
masters.sort_values(by = 'ms_quant', ascending = False)

В этом датасете имеем уже 89 регионов (таковы сегодняшние реалии). Уберём 4 новых региона: ДНР, ЛНР, Херсонскую и Запорожскую области. Вдобавок, придётся снова привести названия регионов к соответсвующим названиям в датасете regions_pools.

In [ ]:
masters[masters['region'] == 'Запорожская область']

In [ ]:
masters.drop([88, 87, 86, 83], inplace = True)
masters

In [ ]:
masters['region'] = masters['region'].str.replace(' автономный округ', ' АО')
masters['region'] = masters['region'].str.replace(' автономная область', ' АО')
masters['region'] = masters['region'].str.replace('Ханты-Мансийский АО — Югра', 'Ханты-Мансийский АО')
masters[masters['region'].str.contains('АО')]
masters

In [ ]:
regions_swimming_1 = pd.merge(regions_pools, masters, left_on = 'region', right_on = 'region', how = 'left')
regions_swimming_2 = pd.merge(regions_pools, masters, left_on = 'region', right_on = 'region', how = 'right')

In [ ]:
regions_swimming_1[regions_swimming_1['ms_quant'].isna()]

In [ ]:
regions_swimming_2[regions_swimming_2['population'].isna()]

In [ ]:
masters['region'] = masters['region'].str.replace('город ', '')
masters.at[42, 'region'] = 'Кемеровская область'

Теперь всё в порядке, все 85 субъектов с данными:

In [ ]:
regions_swimming = pd.merge(regions_pools, masters, left_on = 'region', right_on = 'region', how = 'inner')
regions_swimming

### Регионы с условно развитым плаванием

И новый показатель: количество Мастеров спорта по плаванию на 100 тысяч жителей региона. На основании этого показателя, если его значение будет не меньше среднего по России, будем считать регион имеющим развитое спортивное плавание. Объективных причин для такой кластеризации нет, но в нашем случае едва ли можно придумать объективный критерий в принципе.

In [ ]:
regions_swimming['ms_density'] = round(regions_swimming['ms_quant'] / regions_swimming['population'] * 100000, 2)
regions_swimming

In [ ]:
avg_ms_density = regions_swimming['ms_density'].mean()
avg_ms_density

In [ ]:
developed_regions = regions_swimming[regions_swimming['ms_density'] >= avg_ms_density]
developed_regions.sort_values(by = 'ms_density', ascending = False)

In [ ]:
print(f"Согласно нашему критерию, в России {developed_regions['region'].count()} региона с развитым плаванием.")

## Визуализации и корреляции

Теперь у нас много данных и большое поле для действий.

- Во-первых, можно оценить взаимосвязь количества Мастеров спорта по плаванию в регионе (прогнозируемая переменная) с другими переменными (независимые переменные).

- Во-вторых, хорошо бы посмотреть, какие из перменных (население, бассейны 50 метров, общее число бассейнов, плотность бассейнов или плотность 50-метровых бассейнов) наиболее кореллируют с числом Мастеров спорта по плаванию в регионе.

- Наконец, круто было бы получить прогноз числа мастеров спорта, исходя из значимых перменных.

### 1) Население - МС по плаванию

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)
x = regions_swimming['population'] / 1000000
y = regions_swimming['ms_quant']

ax.scatter(x, y, color = 'b')
ax.set_xlabel('Население, млн', fontsize = 12)
ax.set_ylabel('Количество МС по плаванию', fontsize = 12)
ax.set_title('Scatter plot: население региона — МС по плаванию', weight = 'bold', fontsize = 14)
ax.set_facecolor('whitesmoke')
ax.grid(True)
z = np.polyfit(regions_swimming['population'] / 1000000, regions_swimming['ms_quant'], 1)
p = np.poly1d(z)
ax.plot(x, p(x), color = "purple")

In [ ]:
correlation_ms_population = regions_swimming['ms_quant'].corr(regions_swimming['population'])
print(correlation_ms_population)

### 2) Бассейны - МС по плаванию

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
x1, x2 = regions_swimming['total'], regions_swimming['50m']
y = regions_swimming['ms_quant']

ax1.scatter(x1, y, color = 'b')
ax1.set_xlabel('Количество бассейнов', fontsize = 12)
ax1.set_ylabel('Количество МС по плаванию', fontsize = 12)
ax1.set_title('Scatter plot: количество бассейнов — МС по плаванию', weight = 'bold', fontsize = 14)
ax1.set_facecolor('whitesmoke')
ax1.grid(True)
z = np.polyfit(regions_swimming['total'], regions_swimming['ms_quant'], 1)
p = np.poly1d(z)
ax1.plot(x1, p(x1), color = "purple")

ax2.scatter(x2, y, color = 'b')
ax2.set_xlabel('Количество 50 м бассейнов', fontsize = 12)
ax2.set_ylabel('Количество МС по плаванию', fontsize = 12)
ax2.set_title('Scatter plot: количество 50 м бассейнов — МС по плаванию', weight = 'bold', fontsize = 14)
ax2.set_facecolor('whitesmoke')
ax2.grid(True)
z = np.polyfit(regions_swimming['50m'], regions_swimming['ms_quant'], 1)
p = np.poly1d(z)
ax2.plot(x2, p(x2), color = "purple")

In [ ]:
correlation_ms_total = regions_swimming['ms_quant'].corr(regions_swimming['total'])
print(correlation_ms_total)

In [ ]:
correlation_ms_50m = regions_swimming['ms_quant'].corr(regions_swimming['50m'])
print(correlation_ms_50m)

### 3) Плотности - МС по плаванию

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
x1, x2 = regions_swimming['pools_density'], regions_swimming['50m_pools_density']
y = regions_swimming['ms_quant']

ax1.scatter(x1, y, color = 'b')
ax1.set_xlabel('Плотность бассейнов, басс./100000 тыс. жителей', fontsize = 12)
ax1.set_ylabel('Количество МС по плаванию', fontsize = 12)
ax1.set_title('Scatter plot: плотность бассейнов — МС по плаванию', weight = 'bold', fontsize = 14)
ax1.set_facecolor('whitesmoke')
ax1.grid(True)
z = np.polyfit(regions_swimming['pools_density'], regions_swimming['ms_quant'], 1)
p = np.poly1d(z)
ax1.plot(x1, p(x1), color = "purple")

ax2.scatter(x2, y, color = 'b')
ax2.set_xlabel('Плотность 50 м бассейнов, 50 м басс. /100000 тыс. жителей', fontsize = 12)
ax2.set_ylabel('Количество МС по плаванию', fontsize = 12)
ax2.set_title('Scatter plot: плотность 50 м бассейнов — МС по плаванию', weight = 'bold', fontsize = 14)
ax2.set_facecolor('whitesmoke')
ax2.grid(True)
z = np.polyfit(regions_swimming['50m_pools_density'], regions_swimming['ms_quant'], 1)
p = np.poly1d(z)
ax2.plot(x2, p(x2), color = "purple")

In [ ]:
correlation_ms_density = regions_swimming['ms_quant'].corr(regions_swimming['pools_density'])
print(correlation_ms_density)

In [ ]:
correlation_ms_50m_density = regions_swimming['ms_quant'].corr(regions_swimming['50m_pools_density'])
print(correlation_ms_50m_density)

Соберём корреляции в кучу:

In [ ]:
correlations = regions_swimming[['ms_quant', 'population', 'total', '50m', 'pools_density', '50m_pools_density', 'ms_density']].corr()

correlations.style.format(precision=2)\
  .background_gradient(cmap='coolwarm')

### Промежуточный вывод

Выходит, что при анализе всех 85 регионов наиболее наибольшую корреляцию число Мастеров спорта имеет, как это ни банально, с общим числом бассейно в регионе. Даже немного грустно видеть такой результат, и особенно — видеть мизерную корреляцию с плотностью бассейнов. Ну да никто и не обещал, что сможем наблюдать связь с этим показателем.

Если мы выбираем плотность мастеров спорта как метрику, отражающую развитие региона в спортивном плавании, то сильной коррелиции не удаётся увидеть ни с каким из показателей, что тоже не самый радостный результат.

Думаю, что можно попробовать построить предсказтельную модель числа Мастеров спорта по плаванию на основании двух показателей: населения и общего числа бассейнов. 

### 4) Предсказательная модель

In [ ]:
X = regions_swimming.loc[:, ['population', 'total']]
y = regions_swimming['ms_quant']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print("Mean squared error (MSE):  %.2f" % mean_squared_error(y_test, y_pred))
print("Mean absolute error (MAE):  %.2f" % mean_absolute_error(y_test, y_pred))
print("Coefficient of determination (R^2): %.2f" % r2_score(y_test, y_pred))
print(f'Coefficients: {regr.coef_[0]:.5f}, {regr.coef_[1]:.5f}')

Модель даёт лишь уодвлетворительный результат: R^2 маловат. 

Коэффициент при численности населения почти нулевой, однако это неудивительно: размерность переменных совершенно разная, поэтому такой результат более чем адекватен.

In [ ]:
def predict_ms(population, pools):
    ms_quant = population * 0.00003 + pools * 0.76279
    return round(ms_quant)

population = int(input('Введите число жителей в вашем регионе:'))
pools = int(input('Введите число бассейнов в вашем регионе:'))
print(f'Предполагаемое число Мастеров Спорта по плаванию в регионе: {predict_ms(population, pools)}')

Оставляю эту функцию здесь, но её же добавил и в приложение streamlit.

## Карта 

Теперь создадим карту на основании плотности Мастеров Спорта в регионе.

In [ ]:
#выгружаю датасет локально, чтобы впоследствии сделать демонстрацию карты в Streamlit

regions_swimming.to_file("regions_swimming.json", driver="GeoJSON")

In [ ]:
m = folium.Map(location=[64.6863136, 97.7453061], zoom_start=3)

max_count = regions_swimming['ms_density'].max()
min_count = regions_swimming['ms_density'].min()
colormap = cm.linear.PuBuGn_09.scale(min_count, max_count)
colormap.add_to(m)

def style_function(feature):
    regions_swimming = feature['properties']['ms_density']
    return {
        'fillColor': colormap(regions_swimming),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

folium.GeoJson(
    regions_swimming,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['region', 'ms_quant', 'ms_density'],
        aliases=['Регион:', 'Количество МС по плаванию:', 'Число МС по плаванию на 100 тыс. жителей:'],
        localize=True,
    )
).add_to(m)

m

# Часть 4. SQL

Удалим столбец с геометрией: в базе данных он нам не понадобится.

In [ ]:
del regions_swimming['geometry']

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///regions_swimming.db")
regions_swimming.to_sql('swimming_data', engine, if_exists='replace') 

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///regions_swimming.db

Найдём топ-15 регионов по количеству МС по плаванию:

In [ ]:
%%sql
SELECT region, ms_quant 
FROM swimming_data 
ORDER BY ms_quant DESC
LIMIT 15

Теперь по плотности:

In [ ]:
%%sql
SELECT region, ms_density, ms_quant, total, population
FROM swimming_data 
ORDER BY ms_density DESC
LIMIT 5

Внезапно, Москвы здесь не оказалось, хотя есть Санкт-Петербург.

Посмотрим на статистику по федеральным округам:

In [ ]:
%%sql
SELECT federal_district, 
SUM(ms_quant) as sum_ms,
SUM(total) as sum_total_pools,
ROUND(AVG(ms_density), 2) as avg_ms_density, 
ROUND(AVG(ms_quant), 2) as avg_ms_quant, 
ROUND(AVG(total), 2) as avg_total_pools
FROM swimming_data 
GROUP BY federal_district
ORDER BY sum_ms DESC

Выберем те регионы, где число бассейнов и число МС больше среднего по России

In [ ]:
%%sql
SELECT ROUND(AVG(ms_quant), 2), ROUND(AVG(total), 2) FROM swimming_data

In [ ]:
%%sql
SELECT region, population, ms_quant, total as pools_quant FROM swimming_data
WHERE ms_quant > (SELECT AVG(ms_quant) FROM swimming_data) and  total > (SELECT AVG(total) FROM swimming_data)
ORDER BY population DESC

Наконец, посмотрим на топ-10 регионов, где наиболее эффективно развивается плавание, то есть в которых число Мастеров спорта на один бассейн региона наибольшее:

In [ ]:
%%sql
SELECT *, ROUND(ms_quant / CAST(total AS FLOAT), 2) AS efficiency FROM swimming_data
ORDER BY efficiency DESC
LIMIT 10

# Итоги

В результате анализа удалось прийти к изначально интуитивному выводу, что больше всего плавание развито там, где больше всего бассейнов. Но получены и несколько, по моему мнению, интересных результатов и любопытных наблюдений.

- Интересный вывод 1: в среднем на 100 тысяч жителей России приходится 2 Мастера спорта по плаванию.

- Интересный вывод 2: Пензенская область — абсолютный лидер по "плотности" Мастеров спорта по плаванию.

- Интересный вывод 3: во всех показателях, отнормированных по населению или по числу бассейнов (речь про "эффективность"), Санкт-Петербург опережает Москву.